In [ ]:
!pip install feedparser -q

In [ ]:
import feedparser
from tqdm import tqdm

In [ ]:
    def _process(item):
        item = {
            'title': item.get("title", ""),
            'published date': item.get("published", ""),
            'url': item.get("link", ""),
            'publisher': item.get("source", " ")
        }
        return item

    def get_news(key):
        if key != '':
            url = key
            return list(map(_process, feedparser.parse(url).entries[:1000]))

In [ ]:
def t_url(keyword,date_after, date_before):
  return "https://news.google.com/rss/search?hl=id&gl=ID&ceid=ID:id&q="+keyword+"+after:"+"-".join(date_after)+"+before:"+"-".join(date_before)

In [ ]:
def gen_url_diff(keyword,min_dt, max_dt):
  all_url = []

  # tahun kemarin
  for b in range(12):
    if b<min_dt[1]:
      continue
    for h in range(30):
      if b==min_dt[1] and h<min_dt[2]:
        continue
      #print(x," ",y)
      all_url.append( t_url(keyword, [str(min_dt[0]), str(b), str(h)], [str(min_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(min_dt[0]), str(b), str(31)], ([str(min_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))

  # tahun skrg
  for b in range(12):
    if b>max_dt[1]:
      continue
    for h in range(30):
      if b==max_dt[1] and h>max_dt[2]:
        continue
      all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(h)], [str(max_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(31)], ([str(max_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))


  return all_url

In [ ]:
def gen_url(keyword,min_dt, max_dt):
  all_url = []

  for b in range(1, 13):
    if b<min_dt[1]:
      continue
    if b>max_dt[1]:
      continue
    for h in range(1, 31):
      if b==min_dt[1] and h<min_dt[2]:
        continue
      if b==max_dt[1] and h>max_dt[2]:
        continue
      all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(h)], [str(max_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(31)], ([str(max_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))
    if max_dt[2] != 31:
      del all_url[-1]
  return all_url

In [ ]:
def run(keyword,min_date,max_date):
  all_url = gen_url_diff(keyword,min_date,max_date) if max_date[0]!=min_date[0] else gen_url(keyword,min_date,max_date)
  
  result = []
  for url in tqdm(all_url):
    result.append(get_news(url))

  return result

In [ ]:
keyword   = "tkbm"
min_date  = [2021,9,1]  # t-b-h
max_date  = [2021,9,10] # t-b-h


In [ ]:
crawl = run(keyword, min_date, max_date)

crawl_list=[]
count=0
for x in crawl:
  count=count+len(x)
  if x!=[]:
    for key, value in x[0].items():
      temp = [key,value]
      crawl_list.append(temp)
count

In [ ]:
crawl_list=[]
count=0

for x in crawl:
  count=count+len(x)
  if x!=[]:
    for y in x:
      temp=[]
      for key, value in y.items():
        temp.append(value)
      crawl_list.append(temp)
count

In [ ]:
import pandas as pd
crawl_df=pd.DataFrame(crawl_list)
crawl_df.columns=['Judul',"Tanggal","Link","Publisher"]
crawl_df.info()


In [ ]:
crawl_df

In [ ]:
path="/content/drive/MyDrive/UAS/TKBM/"
#crawl_df.to_csv(path+"crawl.csv",index=False)